In [1]:
import os
from datetime import datetime

class Libro:
    """Clase que representa un libro en la biblioteca"""

    def __init__(self, titulo, autor, año, estado="disponible"):
        """
        Constructor de la clase Libro

        Args:
            titulo (str): Título del libro
            autor (str): Autor del libro
            año (int): Año de publicación
            estado (str): Estado del libro (disponible/prestado)
        """
        self._titulo = titulo
        self._autor = autor
        self._año = año
        self._estado = estado

    # Métodos getters
    def get_titulo(self):
        return self._titulo

    def get_autor(self):
        return self._autor

    def get_año(self):
        return self._año

    def get_estado(self):
        return self._estado

    # Métodos setters
    def set_titulo(self, titulo):
        self._titulo = titulo

    def set_autor(self, autor):
        self._autor = autor

    def set_año(self, año):
        self._año = año

    def set_estado(self, estado):
        if estado in ["disponible", "prestado"]:
            self._estado = estado
        else:
            raise ValueError("El estado debe ser 'disponible' o 'prestado'")

    def marcar_como_prestado(self):
        """Marca el libro como prestado"""
        if self._estado == "prestado":
            raise ValueError(f"El libro '{self._titulo}' ya está prestado")
        self._estado = "prestado"

    def devolver(self):
        """Marca el libro como disponible"""
        if self._estado == "disponible":
            raise ValueError(f"El libro '{self._titulo}' no estaba prestado")
        self._estado = "disponible"

    def __str__(self):
        """Representación en cadena del libro"""
        return f"Título: {self._titulo}, Autor: {self._autor}, Año: {self._año}, Estado: {self._estado}"

    def to_file_format(self):
        """Convierte el libro a formato para archivo"""
        return f"{self._titulo}|{self._autor}|{self._año}|{self._estado}|libro"


class LibroDigital(Libro):
    """Clase que representa un libro digital, hereda de Libro"""

    def __init__(self, titulo, autor, año, formato, estado="disponible"):
        """
        Constructor de la clase LibroDigital

        Args:
            titulo (str): Título del libro
            autor (str): Autor del libro
            año (int): Año de publicación
            formato (str): Formato del libro digital (PDF, ePub, etc.)
            estado (str): Estado del libro (disponible/prestado)
        """
        super().__init__(titulo, autor, año, estado)
        self._formato = formato

    def get_formato(self):
        return self._formato

    def set_formato(self, formato):
        self._formato = formato

    def __str__(self):
        """Representación en cadena del libro digital (polimorfismo)"""
        return f"Título: {self._titulo}, Autor: {self._autor}, Año: {self._año}, Estado: {self._estado}, Formato: {self._formato}"

    def to_file_format(self):
        """Convierte el libro digital a formato para archivo"""
        return f"{self._titulo}|{self._autor}|{self._año}|{self._estado}|digital|{self._formato}"


class Biblioteca:
    """Clase que gestiona una colección de libros"""

    def __init__(self, archivo="biblioteca.txt"):
        """
        Constructor de la biblioteca

        Args:
            archivo (str): Nombre del archivo para persistencia
        """
        self._libros = []
        self._archivo = archivo
        self.cargar_libros()

    def cargar_libros(self):
        """Carga los libros desde el archivo"""
        try:
            if os.path.exists(self._archivo):
                with open(self._archivo, 'r', encoding='utf-8') as file:
                    for linea in file:
                        linea = linea.strip()
                        if linea:
                            partes = linea.split('|')
                            if len(partes) >= 5:
                                titulo, autor, año, estado, tipo = partes[:5]
                                try:
                                    año = int(año)
                                    if tipo == "digital" and len(partes) >= 6:
                                        formato = partes[5]
                                        libro = LibroDigital(titulo, autor, año, formato, estado)
                                    else:
                                        libro = Libro(titulo, autor, año, estado)
                                    self._libros.append(libro)
                                except ValueError:
                                    print(f"Error al procesar la línea: {linea}")
                print(f"Se cargaron {len(self._libros)} libros desde {self._archivo}")
        except Exception as e:
            print(f"Error al cargar los libros: {e}")

    def guardar_libros(self):
        """Guarda los libros en el archivo"""
        try:
            with open(self._archivo, 'w', encoding='utf-8') as file:
                for libro in self._libros:
                    file.write(libro.to_file_format() + '\n')
            print(f"Libros guardados exitosamente en {self._archivo}")
        except Exception as e:
            print(f"Error al guardar los libros: {e}")

    def agregar_libro(self, libro):
        """
        Agrega un libro a la biblioteca

        Args:
            libro (Libro): El libro a agregar
        """
        if not isinstance(libro, Libro):
            raise TypeError("El objeto debe ser una instancia de Libro")

        # Verificar si ya existe un libro con el mismo título
        for lib in self._libros:
            if lib.get_titulo().lower() == libro.get_titulo().lower():
                raise ValueError(f"Ya existe un libro con el título '{libro.get_titulo()}'")

        self._libros.append(libro)
        print(f"Libro '{libro.get_titulo()}' agregado exitosamente")

    def eliminar_libro(self, titulo):
        """
        Elimina un libro por su título

        Args:
            titulo (str): Título del libro a eliminar
        """
        for i, libro in enumerate(self._libros):
            if libro.get_titulo().lower() == titulo.lower():
                libro_eliminado = self._libros.pop(i)
                print(f"Libro '{libro_eliminado.get_titulo()}' eliminado exitosamente")
                return

        raise ValueError(f"No se encontró un libro con el título '{titulo}'")

    def listar_libros_disponibles(self):
        """Lista todos los libros disponibles"""
        libros_disponibles = [libro for libro in self._libros if libro.get_estado() == "disponible"]

        if not libros_disponibles:
            print("No hay libros disponibles")
            return

        print("\n=== LIBROS DISPONIBLES ===")
        for i, libro in enumerate(libros_disponibles, 1):
            print(f"{i}. {libro}")

    def listar_todos_los_libros(self):
        """Lista todos los libros de la biblioteca"""
        if not self._libros:
            print("La biblioteca está vacía")
            return

        print(f"\n=== TODOS LOS LIBROS ({len(self._libros)}) ===")
        for i, libro in enumerate(self._libros, 1):
            print(f"{i}. {libro}")

    def buscar_libro(self, titulo):
        """
        Busca un libro por su título

        Args:
            titulo (str): Título del libro a buscar

        Returns:
            Libro: El libro encontrado o None si no existe
        """
        for libro in self._libros:
            if libro.get_titulo().lower() == titulo.lower():
                return libro
        return None

    def marcar_como_prestado(self, titulo):
        """
        Marca un libro como prestado

        Args:
            titulo (str): Título del libro a prestar
        """
        libro = self.buscar_libro(titulo)
        if libro is None:
            raise ValueError(f"No se encontró un libro con el título '{titulo}'")

        try:
            libro.marcar_como_prestado()
            print(f"El libro '{titulo}' ha sido marcado como prestado")
        except ValueError as e:
            raise ValueError(str(e))

    def devolver_libro(self, titulo):
        """
        Devuelve un libro prestado

        Args:
            titulo (str): Título del libro a devolver
        """
        libro = self.buscar_libro(titulo)
        if libro is None:
            raise ValueError(f"No se encontró un libro con el título '{titulo}'")

        try:
            libro.devolver()
            print(f"El libro '{titulo}' ha sido devuelto y está disponible")
        except ValueError as e:
            raise ValueError(str(e))

    def obtener_estadisticas(self):
        """Muestra estadísticas de la biblioteca"""
        total = len(self._libros)
        disponibles = len([l for l in self._libros if l.get_estado() == "disponible"])
        prestados = total - disponibles
        digitales = len([l for l in self._libros if isinstance(l, LibroDigital)])

        print(f"\n=== ESTADÍSTICAS DE LA BIBLIOTECA ===")
        print(f"Total de libros: {total}")
        print(f"Libros disponibles: {disponibles}")
        print(f"Libros prestados: {prestados}")
        print(f"Libros digitales: {digitales}")


def mostrar_menu():
    """Muestra el menú principal"""
    print("\n" + "="*40)
    print("--- GESTOR DE BIBLIOTECA ---")
    print("="*40)
    print("1. Agregar libro físico")
    print("2. Agregar libro digital")
    print("3. Eliminar libro")
    print("4. Ver todos los libros")
    print("5. Ver libros disponibles")
    print("6. Buscar libro")
    print("7. Marcar libro como prestado")
    print("8. Devolver libro")
    print("9. Ver estadísticas")
    print("10. Salir")
    print("-"*40)


def main():
    """Función principal del programa"""
    print("¡Bienvenido al Sistema de Gestión de Biblioteca!")

    # Crear instancia de la biblioteca
    biblioteca = Biblioteca()

    while True:
        mostrar_menu()

        try:
            opcion = input("Elige una opción (1-10): ").strip()

            if opcion == "1":
                # Agregar libro físico
                print("\n--- AGREGAR LIBRO FÍSICO ---")
                titulo = input("Título: ").strip()
                autor = input("Autor: ").strip()
                año = int(input("Año de publicación: "))

                libro = Libro(titulo, autor, año)
                biblioteca.agregar_libro(libro)

            elif opcion == "2":
                # Agregar libro digital
                print("\n--- AGREGAR LIBRO DIGITAL ---")
                titulo = input("Título: ").strip()
                autor = input("Autor: ").strip()
                año = int(input("Año de publicación: "))
                formato = input("Formato (PDF, ePub, etc.): ").strip()

                libro = LibroDigital(titulo, autor, año, formato)
                biblioteca.agregar_libro(libro)

            elif opcion == "3":
                # Eliminar libro
                print("\n--- ELIMINAR LIBRO ---")
                titulo = input("Título del libro a eliminar: ").strip()
                biblioteca.eliminar_libro(titulo)

            elif opcion == "4":
                # Ver todos los libros
                biblioteca.listar_todos_los_libros()

            elif opcion == "5":
                # Ver libros disponibles
                biblioteca.listar_libros_disponibles()

            elif opcion == "6":
                # Buscar libro
                print("\n--- BUSCAR LIBRO ---")
                titulo = input("Título del libro a buscar: ").strip()
                libro = biblioteca.buscar_libro(titulo)
                if libro:
                    print(f"Libro encontrado: {libro}")
                else:
                    print(f"No se encontró un libro con el título '{titulo}'")

            elif opcion == "7":
                # Marcar como prestado
                print("\n--- MARCAR LIBRO COMO PRESTADO ---")
                titulo = input("Título del libro a prestar: ").strip()
                biblioteca.marcar_como_prestado(titulo)

            elif opcion == "8":
                # Devolver libro
                print("\n--- DEVOLVER LIBRO ---")
                titulo = input("Título del libro a devolver: ").strip()
                biblioteca.devolver_libro(titulo)

            elif opcion == "9":
                # Ver estadísticas
                biblioteca.obtener_estadisticas()

            elif opcion == "10":
                # Salir
                print("\n--- GUARDANDO DATOS ---")
                biblioteca.guardar_libros()
                print("¡Gracias por usar el Sistema de Gestión de Biblioteca!")
                break

            else:
                print("Opción inválida. Por favor seleccione una opción del 1 al 10.")

        except ValueError as e:
            print(f"Error: {e}")
        except Exception as e:
            print(f"Error inesperado: {e}")

        # Pausa antes de mostrar el menú nuevamente
        input("\nPresione Enter para continuar...")


if __name__ == "__main__":
    main()

¡Bienvenido al Sistema de Gestión de Biblioteca!

--- GESTOR DE BIBLIOTECA ---
1. Agregar libro físico
2. Agregar libro digital
3. Eliminar libro
4. Ver todos los libros
5. Ver libros disponibles
6. Buscar libro
7. Marcar libro como prestado
8. Devolver libro
9. Ver estadísticas
10. Salir
----------------------------------------


KeyboardInterrupt: Interrupted by user